In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB_PROJECT"] = "xmen"

In [2]:
import sys
sys.path.append('../scripts')

In [3]:
import openai
from pathlib import Path
import pandas as pd
from tqdm.auto import tqdm

from transformers import AutoModelForSeq2SeqLM,Text2TextGenerationPipeline
from evaluation import error_analysis, get_scores
from dataset import load_data, get_dataloader
from transformers_util import get_training_args, get_tokenizer

In [4]:
import hydra
from hydra import compose, initialize

hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(config_path=Path('..'), job_name='foo', version_base='1.1')
config = compose(config_name='experiment.yaml')

In [5]:
openai.api_key = config.api_key

In [6]:
BATCH_SIZE = 32
SPLIT = 'test'

# Load Data

In [15]:
training_args = get_training_args(config, report_to="none")
tokenizer = get_tokenizer(config)

base_path = Path('..')
train_df, val_df, test_df = load_data(base_path / config.data.cnf_tsv_path, base_path / config.data.controls_tsv_path)
train_dataset, val_dataset, test_dataset = get_dataloader(train_df, val_df, test_df, tokenizer)

/home/Florian.Borchert/miniconda3/envs/ellipses/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [16]:
if SPLIT == 'valid':
    df = val_df
elif SPLIT == 'test':
    df = test_df

In [17]:
output_path = Path(f'results_{SPLIT}')
output_path.mkdir(exist_ok=True)

In [18]:
resolutions = list(df.full_resolution)
samples = list(df.raw_sentence)

# Initialize Model

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("../data/ellipses/model")

# TOP 1 generation from seq-to-seq model

In [ ]:
%%time
pipeline = Text2TextGenerationPipeline(model=model, tokenizer=tokenizer, max_length=config.generation_max_length, device=0, batch_size=BATCH_SIZE)

predictions = pipeline(samples)

errors = error_analysis([prediction["generated_text"] for prediction in predictions], resolutions, samples)

In [ ]:
errors.error_type.value_counts()# / len(errors)

In [ ]:
errors.to_parquet(output_path / 'results_top1.parquet')

In [ ]:
%%time
get_scores(errors, SPLIT)

# Best out of TOP k predictions from seq-to-seq model

In [ ]:
from evaluation import relative_edit_distance

def evaluate_top_k(model, tokenizer, data, beams, generation_max_length=config.generation_max_length, debug=False):
    pipeline = Text2TextGenerationPipeline(
        model=model, tokenizer=tokenizer, max_length=generation_max_length, num_beams=beams, num_return_sequences=beams, device=0, batch_size=BATCH_SIZE)

    originals = list(data.raw_sentence)
    resolutions = list(data.full_resolution)
    outputs = pipeline(originals)

    predictions = []
    for i, resolution in enumerate(resolutions):
        generations = [entry['generated_text'] for entry in outputs[i]]
        scores = [relative_edit_distance(gen, resolution, originals[i]) for gen in generations]
        if debug and max(scores) == 1 and scores.index(max(scores)) != 0:
            print(generations)
            print(scores.index(max(scores)))
            print(resolution)
        predictions.append(generations[scores.index(max(scores))])

    errors = error_analysis(predictions, resolutions, list(data.raw_sentence))

    return errors, outputs

In [ ]:
%%time
k = 5
errors_k, outputs_k = evaluate_top_k(model, tokenizer, val_df, k)

In [ ]:
errors_k.error_type.value_counts() / len(errors_k)

In [ ]:
results_k = pd.concat([errors_k, pd.Series(outputs_k, name='outputs_k')], axis=1)

In [ ]:
results_k.to_parquet(output_path / f'results_top{k}.parquet')

In [ ]:
%%time
get_scores(results_k, SPLIT)

# Using ChatGPT/GPT to determine the best fit of the top k options

## Prompts

In [7]:
def generate_multiple_choice_prompt(original, predictions):
    # Das folgende Problem wurde von Yann LeCun gestellt, der sehr an der Kompetenz von Künstlicher Intelligenz, wie dir, zweifelt: 
    beginning = "Dir werden im Folgenden ein Satz gezeigt, welcher sogennannte Koordinationsellipsen enthält. Das Ziel ist es diese zu aufzulösen. Ein Beispiel wäre 'Ibrutinib, ein Inhibitor der Bruton-Tyrosinkinase (BTK), ist in Deutschland als Erstlinien- und Rezidivtherapiee in der CLL zugelassen.' Die richtige Auflösung wäre 'Ibrutinib, ein Inhibitor der Bruton-Tyrosinkinase (BTK), ist in Deutschland als Erstlinientherapie und Rezidivtherapiee in der CLL zugelassen.' Dir werden zu den Beispielen Antwortmöglichkeiten gegeben und du sollst dann entscheiden, welche dieser Optionen die Koordinationsellipsen korrekt auflöst.\n\n"
    original = f"Der originale Satz: '{original}'\n\n"
    answers = "Deine Antwortmöglichkeiten:\n" + "".join(f"{i+1}) '{prediction}'\n" for i, prediction in enumerate(predictions))
    end = "\nWelche Antwort ist die richtige? Antworte nur mit der Zahl und keiner Erklärung"

    return beginning + original + answers + end

In [8]:
def generate_prompt_true_false(original, predictions):
    beginning = "Sie haben ein Modell entwickelt, das Koordinationsellipsen in Sätzen erkennt und auflöst. Das Modell gibt fünf verschiedene Versionen des ursprünglichen Satzes zurück, wobei die erste Version die wahrscheinlichste ist. Bitte lesen Sie sich die erste Version des Satzes sorgfältig durch und entscheiden Sie, ob diese Version korrekt ist und den ursprünglichen Satz mit aufgelösten Koordinationsellipsen wiedergibt. Bitte antworten Sie mir nur 'Ja' oder 'Nein' und keiner Erklärung!\n\n"
    original = f"Ursprünglicher Satz: '{original}'\n\n"
    answer = f"Erste Version: '{predictions[0]}'\n\n"

    return beginning + original + answer

In [9]:
def generate_prompt_other_options(original, predictions):
    beginning = "Sie haben entschieden, dass die erste Version des Satzes, die vom Modell als die wahrscheinlichste ausgewählt wurde, nicht korrekt ist und den ursprünglichen Satz mit aufgelösten Koordinationsellipsen nicht vollständig wiedergibt. Das Modell gibt vier weitere Versionen des Satzes zurück, die als die nächst wahrscheinlichsten Versionen ausgewählt wurden. Bitte lesen Sie sich diese vier Versionen sorgfältig durch und wählen Sie die Version aus, die Ihrer Meinung nach am besten den ursprünglichen Satz mit aufgelösten Koordinationsellipsen wiedergibt.\n\n"
    answers = "" + "".join(f"{i+1} - '{prediction}'\n" for i, prediction in enumerate(predictions[1:]))
    end = "\nBitte antworten Sie nur mit der richtigen Nummer und ohne Erklärung."
    return beginning + answers + end

In [10]:
def generate_best_fit_prompts(samples, outputs, ground_truths, prompt_fn):
    prompt_df = []
    for i, sample in enumerate(samples):
        generations = [entry['generated_text'] for entry in outputs[i]]
        ground_truth = ground_truths[i]
        true_index = generations.index(ground_truth) if ground_truth in generations else -1
        
        prompt_df.append({
            'input' : sample,
            'generations' : generations,
            'true_index' : true_index,
            'prompt' : prompt_fn(sample, generations)
        })
    
    return pd.DataFrame(prompt_df)

## OpenAI API calls

In [11]:
from retry import retry
import logging
logging.basicConfig()

In [12]:
MODEL = "gpt-4-0125-preview"
#MODEL = "gpt-3.5-turbo-1106"

In [13]:
@retry(tries=3, delay=2)
def get_openai_response_chatgpt(messages):
    return openai.ChatCompletion.create(
        model=MODEL,
        messages=[
            {"role": role, "content": text} for (role, text) in messages
        ],
        temperature=0,
        max_tokens=100,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )["choices"][0]["message"]["content"]

## Query GPT-3.5 to pick 1 from k options

In [19]:
results_5 = pd.read_parquet(output_path / f'results_top5.parquet')

In [21]:
from tqdm.auto import tqdm
import re

def call_api_best_fit(prompt_df, skip_if_1st_unchanged=True, debug=False):
    predictions = []

    for i, row in tqdm(list(prompt_df.iterrows())):
        generations = row.generations
        sample = row.input
        prompt = row.prompt
        status = 'success'
        
        if skip_if_1st_unchanged and generations[0] == sample:
            if debug:
                print(f'{i}) answer: {0}')
                print('--------------------------------')
            predictions.append({'status': 'skipped', 'prediction' : generations[0], 'index' : 0, 'answer' : None})

        else:
            message = [("user", prompt)]
            answer = get_openai_response_chatgpt(message)

            numbers = re.findall(r'\d+', answer)
            if len(numbers) > 1:
                if debug:
                    print(f'more numbers than expected {numbers}')
                status = 'error_multiple'
                index = numbers[0]
            if len(numbers) == 0:
                if debug:
                    print(f'no numbers found')
                index = 0
                status = 'error_no_numbers'
            else:
                index = int(numbers[0]) - 1
                if index >= len(generations):
                    if debug:
                        print(f'Index is out of bounds. Something went wrong with the API Answer. Defaulting to 0')
                    index = 0
                    status = 'error_out_of_bounds'

            if debug:
                print(f'{i}) answer: {index}')
                print('--------------------------------')
            predictions.append({'status' : status, 'prediction' : generations[index], 'index' : index, 'answer' : answer})
    
    return pd.DataFrame(predictions)

In [22]:
originals_k = list(results_5.original)
resolutions = list(results_5.ground_truth)
outputs_k = list(results_5.outputs_k)

In [23]:
# Prepare prompts
prompts_best_fit = generate_best_fit_prompts(originals_k, outputs_k, resolutions, generate_multiple_choice_prompt)

In [26]:
%%time
#Call OpenAI API
predictions = call_api_best_fit(prompts_best_fit)

  0%|          | 0/1159 [00:00<?, ?it/s]

CPU times: user 2.86 s, sys: 246 ms, total: 3.11 s
Wall time: 9min


In [27]:
prompts2preds_bestfit = pd.concat([prompts_best_fit, predictions], axis=1)

In [28]:
prompts2preds_bestfit.to_parquet(output_path / 'gpt_4_prompts2preds_bestfit.parquet')

In [29]:
errors_best_fit = error_analysis(prompts2preds_bestfit.prediction, resolutions, samples)

In [61]:
errors_best_fit.error_type.value_counts()

tn         597
tp         434
replace     33
insert      26
delete      26
fn          23
complex     11
fp           9
Name: error_type, dtype: int64

In [31]:
%%time
get_scores(errors_best_fit, SPLIT)

CPU times: user 1min 23s, sys: 47.5 ms, total: 1min 23s
Wall time: 1min 25s


{'test/exact_match': 0.8895599654874892,
 'test/gleu': 0.9804849105524535,
 'test/edit_distance_rel': 0.9518765393803748}

## Two-Step Approach: 1. Ask if 1st option is correct 2. choose from options 2-k otherwise

In [32]:
results_5 = pd.read_parquet(output_path / 'results_top5.parquet')
results_5.head()

,pred,ground_truth,original,error_type,index,file,sentence_id,outputs_k
0,Hauptrisikofaktoren für das Auftreten eines Mu...,Hauptrisikofaktoren für das Auftreten eines Mu...,Hauptrisikofaktoren für das Auftreten eines Mu...,tp,0,00_mundhoehlenkarzinom_0002.tsv,1,[{'generated_text': 'Hauptrisikofaktoren für d...
1,Bei chronischem Tabakabusus oder Alkoholabusus...,Bei chronischem Tabakabusus oder Alkoholabusus...,Bei chronischem Tabak- oder Alkoholabusus ist ...,tp,1,00_mundhoehlenkarzinom_0002.tsv,2,[{'generated_text': 'Bei chronischem Tabakabus...
2,Als kurativ intendierte therapeutische Optione...,Als kurativ intendierte therapeutische Optione...,Als kurativ intendierte therapeutische Optione...,tp,40,00_mundhoehlenkarzinom_0098.tsv,2,[{'generated_text': 'Als kurativ intendierte t...
3,"Patienten mit einem unheilbaren Tumorleiden, j...","Patienten mit einem unheilbaren Tumorleiden, j...","Patienten mit einem unheilbaren Tumorleiden, j...",tp,44,00_mundhoehlenkarzinom_0103.tsv,1,[{'generated_text': 'Patienten mit einem unhei...
4,"Patienten mit einem unheilbaren Tumorleiden, j...","Patienten mit einem unheilbaren Tumorleiden, j...","Patienten mit einem unheilbaren Tumorleiden, j...",tp,47,00_mundhoehlenkarzinom_0115.tsv,4,[{'generated_text': 'Patienten mit einem unhei...


In [33]:
top1_classifier_prompts = generate_best_fit_prompts(results_5.original, results_5.outputs_k, results_5.ground_truth, generate_prompt_true_false)
print(top1_classifier_prompts.iloc[0].prompt)

Sie haben ein Modell entwickelt, das Koordinationsellipsen in Sätzen erkennt und auflöst. Das Modell gibt fünf verschiedene Versionen des ursprünglichen Satzes zurück, wobei die erste Version die wahrscheinlichste ist. Bitte lesen Sie sich die erste Version des Satzes sorgfältig durch und entscheiden Sie, ob diese Version korrekt ist und den ursprünglichen Satz mit aufgelösten Koordinationsellipsen wiedergibt. Bitte antworten Sie mir nur 'Ja' oder 'Nein' und keiner Erklärung!

Ursprünglicher Satz: 'Hauptrisikofaktoren für das Auftreten eines Mundhöhlenkarzinoms sind chronischer Tabak- oder Alkoholabusus, wesentlich seltener auch andere Faktoren.'

Erste Version: 'Hauptrisikofaktoren für das Auftreten eines Mundhöhlenkarzinoms sind chronischer Tabakabusus oder Alkoholabusus, wesentlich seltener auch andere Faktoren.'




In [36]:
from tqdm.auto import tqdm
import re

def call_api_top1_classifier(prompt_df, skip_if_1st_unchanged=True, debug=False):
    predictions = []
    for i, row in tqdm(list(prompt_df.iterrows())):
        sample = row.input
        generations = row.generations
        prompt = row.prompt

        if skip_if_1st_unchanged and generations[0] == sample:
            if debug:
                print(f'{i}) answer: {0}')
                print('--------------------------------')
            predictions.append({'status': 'skipped', 'prediction1' : generations[0], 'accept' : True, 'answer' : None})
        else:
            message = [("user", prompt)]
            answer = get_openai_response_chatgpt(message)
            extracted_answer = re.findall(r'Ja|Nein', answer)
            if len(extracted_answer) > 1:
                if debug:
                    print(f'more answers than expected: {answer}. Defaulting to 0')
                    print(f'{i}) answer: {0}')
                    print('--------------------------------')
                predictions.append({'status': 'error_multiple', 'prediction1' : generations[0], 'accept' : True, 'answer' : answer})
            elif len(extracted_answer) == 0:
                if debug:
                    print(f'No answer provided: {answer}. Defaulting to 0')
                    print(f'{i}) answer: {0}')
                    print('--------------------------------')
                predictions.append({'status': 'error_no_answer', 'prediction1' : generations[0], 'accept' : True, 'answer' : answer})
            elif extracted_answer[0] == 'Ja':
                if debug:
                    print(f'{i}) answer: {0}')
                    print('--------------------------------')
                predictions.append({'status': 'success', 'prediction1' : generations[0], 'accept' : True, 'answer' : answer})
            elif extracted_answer[0] == 'Nein':
                if debug:
                    print(f'{i}) answer: != 0')
                    print('--------------------------------')
                predictions.append({'status': 'success', 'prediction1' : None, 'accept' : False, 'answer' : answer})
    return pd.DataFrame(predictions)

In [37]:
predictions_1 = call_api_top1_classifier(top1_classifier_prompts, debug=False)

  0%|          | 0/1159 [00:00<?, ?it/s]

In [38]:
prompt2predictions_1 = pd.concat([top1_classifier_prompts, predictions_1], axis=1)

In [39]:
prompt2predictions_1.to_parquet(output_path / 'prompt2predictions_1.parquet')

In [40]:
prompt2predictions_1.accept.value_counts()

True     1108
False      51
Name: accept, dtype: int64

In [41]:
accepted = prompt2predictions_1[prompt2predictions_1.accept]
len(accepted[accepted.true_index > 0]) / len(accepted)

0.033393501805054154

In [42]:
not_accepted = prompt2predictions_1[~prompt2predictions_1.accept]
len(not_accepted[not_accepted.true_index > 0]) / len(not_accepted)

0.27450980392156865

# Ask GPT to choose from remaining options

In [43]:
results_5 = pd.read_parquet(output_path / 'results_top5.parquet')
results_5.head()

,pred,ground_truth,original,error_type,index,file,sentence_id,outputs_k
0,Hauptrisikofaktoren für das Auftreten eines Mu...,Hauptrisikofaktoren für das Auftreten eines Mu...,Hauptrisikofaktoren für das Auftreten eines Mu...,tp,0,00_mundhoehlenkarzinom_0002.tsv,1,[{'generated_text': 'Hauptrisikofaktoren für d...
1,Bei chronischem Tabakabusus oder Alkoholabusus...,Bei chronischem Tabakabusus oder Alkoholabusus...,Bei chronischem Tabak- oder Alkoholabusus ist ...,tp,1,00_mundhoehlenkarzinom_0002.tsv,2,[{'generated_text': 'Bei chronischem Tabakabus...
2,Als kurativ intendierte therapeutische Optione...,Als kurativ intendierte therapeutische Optione...,Als kurativ intendierte therapeutische Optione...,tp,40,00_mundhoehlenkarzinom_0098.tsv,2,[{'generated_text': 'Als kurativ intendierte t...
3,"Patienten mit einem unheilbaren Tumorleiden, j...","Patienten mit einem unheilbaren Tumorleiden, j...","Patienten mit einem unheilbaren Tumorleiden, j...",tp,44,00_mundhoehlenkarzinom_0103.tsv,1,[{'generated_text': 'Patienten mit einem unhei...
4,"Patienten mit einem unheilbaren Tumorleiden, j...","Patienten mit einem unheilbaren Tumorleiden, j...","Patienten mit einem unheilbaren Tumorleiden, j...",tp,47,00_mundhoehlenkarzinom_0115.tsv,4,[{'generated_text': 'Patienten mit einem unhei...


In [44]:
prompt2predictions_1 = pd.read_parquet(output_path / 'prompt2predictions_1.parquet')
prompt2predictions_1.head()

,input,generations,true_index,prompt,status,prediction1,accept,answer
0,Hauptrisikofaktoren für das Auftreten eines Mu...,[Hauptrisikofaktoren für das Auftreten eines M...,0,"Sie haben ein Modell entwickelt, das Koordinat...",success,Hauptrisikofaktoren für das Auftreten eines Mu...,True,Ja
1,Bei chronischem Tabak- oder Alkoholabusus ist ...,[Bei chronischem Tabakabusus oder Alkoholabusu...,0,"Sie haben ein Modell entwickelt, das Koordinat...",success,Bei chronischem Tabakabusus oder Alkoholabusus...,True,Ja
2,Als kurativ intendierte therapeutische Optione...,[Als kurativ intendierte therapeutische Option...,0,"Sie haben ein Modell entwickelt, das Koordinat...",success,Als kurativ intendierte therapeutische Optione...,True,Ja
3,"Patienten mit einem unheilbaren Tumorleiden, j...","[Patienten mit einem unheilbaren Tumorleiden, ...",0,"Sie haben ein Modell entwickelt, das Koordinat...",success,"Patienten mit einem unheilbaren Tumorleiden, j...",True,Ja
4,"Patienten mit einem unheilbaren Tumorleiden, j...","[Patienten mit einem unheilbaren Tumorleiden, ...",0,"Sie haben ein Modell entwickelt, das Koordinat...",success,"Patienten mit einem unheilbaren Tumorleiden, j...",True,Ja


In [45]:
other_options_prompts = generate_best_fit_prompts(results_5.original, results_5.outputs_k, results_5.ground_truth, generate_prompt_other_options)
print(other_options_prompts.iloc[0].prompt)

Sie haben entschieden, dass die erste Version des Satzes, die vom Modell als die wahrscheinlichste ausgewählt wurde, nicht korrekt ist und den ursprünglichen Satz mit aufgelösten Koordinationsellipsen nicht vollständig wiedergibt. Das Modell gibt vier weitere Versionen des Satzes zurück, die als die nächst wahrscheinlichsten Versionen ausgewählt wurden. Bitte lesen Sie sich diese vier Versionen sorgfältig durch und wählen Sie die Version aus, die Ihrer Meinung nach am besten den ursprünglichen Satz mit aufgelösten Koordinationsellipsen wiedergibt.

1 - 'Nebenrisikofaktoren für das Auftreten eines Mundhöhlenkarzinoms sind chronischer Tabakabusus oder Alkoholabusus, wesentlich seltener auch andere Faktoren.'
2 - 'Hauptrisikofaktoren für das Auftreten eines Mundhöhlenkarzinoms sind chronischer Tabakabusus oder Alkoholabusus, wesentlich seltener und andere Faktoren.'
3 - 'Kopfrisikofaktoren für das Auftreten eines Mundhöhlenkarzinoms sind chronischer Tabakabusus oder Alkoholabusus, wesentl

In [46]:
prompt2predictions_2 = pd.concat([prompt2predictions_1, other_options_prompts.prompt.rename('prompt2')], axis=1)
prompt2predictions_2.head()

,input,generations,true_index,prompt,status,prediction1,accept,answer,prompt2
0,Hauptrisikofaktoren für das Auftreten eines Mu...,[Hauptrisikofaktoren für das Auftreten eines M...,0,"Sie haben ein Modell entwickelt, das Koordinat...",success,Hauptrisikofaktoren für das Auftreten eines Mu...,True,Ja,"Sie haben entschieden, dass die erste Version ..."
1,Bei chronischem Tabak- oder Alkoholabusus ist ...,[Bei chronischem Tabakabusus oder Alkoholabusu...,0,"Sie haben ein Modell entwickelt, das Koordinat...",success,Bei chronischem Tabakabusus oder Alkoholabusus...,True,Ja,"Sie haben entschieden, dass die erste Version ..."
2,Als kurativ intendierte therapeutische Optione...,[Als kurativ intendierte therapeutische Option...,0,"Sie haben ein Modell entwickelt, das Koordinat...",success,Als kurativ intendierte therapeutische Optione...,True,Ja,"Sie haben entschieden, dass die erste Version ..."
3,"Patienten mit einem unheilbaren Tumorleiden, j...","[Patienten mit einem unheilbaren Tumorleiden, ...",0,"Sie haben ein Modell entwickelt, das Koordinat...",success,"Patienten mit einem unheilbaren Tumorleiden, j...",True,Ja,"Sie haben entschieden, dass die erste Version ..."
4,"Patienten mit einem unheilbaren Tumorleiden, j...","[Patienten mit einem unheilbaren Tumorleiden, ...",0,"Sie haben ein Modell entwickelt, das Koordinat...",success,"Patienten mit einem unheilbaren Tumorleiden, j...",True,Ja,"Sie haben entschieden, dass die erste Version ..."


In [57]:
prompt2predictions_2.head().prompt2.iloc[0]

"Sie haben entschieden, dass die erste Version des Satzes, die vom Modell als die wahrscheinlichste ausgewählt wurde, nicht korrekt ist und den ursprünglichen Satz mit aufgelösten Koordinationsellipsen nicht vollständig wiedergibt. Das Modell gibt vier weitere Versionen des Satzes zurück, die als die nächst wahrscheinlichsten Versionen ausgewählt wurden. Bitte lesen Sie sich diese vier Versionen sorgfältig durch und wählen Sie die Version aus, die Ihrer Meinung nach am besten den ursprünglichen Satz mit aufgelösten Koordinationsellipsen wiedergibt.\n\n1 - 'Nebenrisikofaktoren für das Auftreten eines Mundhöhlenkarzinoms sind chronischer Tabakabusus oder Alkoholabusus, wesentlich seltener auch andere Faktoren.'\n2 - 'Hauptrisikofaktoren für das Auftreten eines Mundhöhlenkarzinoms sind chronischer Tabakabusus oder Alkoholabusus, wesentlich seltener und andere Faktoren.'\n3 - 'Kopfrisikofaktoren für das Auftreten eines Mundhöhlenkarzinoms sind chronischer Tabakabusus oder Alkoholabusus, we

In [49]:
def call_api_otheroptions_classifier(prompt_df, debug=False):
    predictions = []
    for i, row in tqdm(list(prompt_df.iterrows())):
        if row.accept:
            if debug:
                print(f'{i} was accepted previously.')
            predictions.append({'status2': 'accept_1', 'prediction2' : row.prediction1, 'answer2' : None})
        else:
            prompt1 = row.prompt
            answer = row.answer
            prompt2 = row.prompt2

            messages = [("user", prompt1), ("assistant", answer), ("user", prompt2)]

            answer2 = get_openai_response_chatgpt(messages)

            numbers = re.findall(r'\d+', answer2)
            if len(numbers) > 1:
                if debug:
                    print(f'more numbers than expected {numbers}')
                predictions.append({'status2': 'error_multiple', 'prediction2' : row.generations[1], 'answer2' : answer2})
            if len(numbers) == 0:
                if debug:
                    print(f'no numbers found')
                predictions.append({'status2': 'no_numbers', 'prediction2' : row.generations[1], 'answer2' : answer2})
            else:
                index = int(numbers[0])
                if index > 4:
                    if debug:
                        print(f'Index is out of bounds. Something went wrong with the API Answer. Defaulting to 0')
                    predictions.append({'status2': 'no_numbers', 'prediction2' : row.generations[1], 'answer2' : answer2})
                else:
                    predictions.append({'status2': 'success', 'prediction2' : row.generations[index], 'answer2' : answer2})
    return pd.DataFrame(predictions)

In [50]:
predictions_2 = call_api_otheroptions_classifier(prompt2predictions_2)

  0%|          | 0/1159 [00:00<?, ?it/s]

In [51]:
result2 = pd.concat([prompt2predictions_2, predictions_2], axis=1)

In [58]:
errors_2 = error_analysis(result2.prediction2, df.full_resolution, result2.input)

In [59]:
errors_2.error_type.value_counts()# / len(errors_2)

tn         596
tp         429
delete      31
fn          30
insert      24
replace     22
complex     17
fp          10
Name: error_type, dtype: int64

In [60]:
%%time
get_scores(errors_2, SPLIT)

CPU times: user 1min 17s, sys: 17 ms, total: 1min 17s
Wall time: 1min 19s


{'test/exact_match': 0.8843830888697153,
 'test/gleu': 0.9796026253080345,
 'test/edit_distance_rel': 0.9432361574177688}

In [ ]:
# Baseline TOP 1 only
errors_top1 = pd.read_parquet(output_path / 'results_top1.parquet')

In [ ]:
errors_top1.error_type.value_counts()

In [ ]:
%%time
get_scores(errors_top1, SPLIT)

In [ ]:
errors_temp = error_analysis(result2.generations.map(lambda l: l[0]), val_df.full_resolution, result2.input)
errors_temp.error_type.value_counts()